# Imports

In [1]:
import pkg_resources
installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
print("TensorFlow:", installed_packages.get("tensorflow"))
print("OpenCV:", installed_packages.get("opencv-python"))
print("NumPy:", installed_packages.get("numpy"))
print("Matplotlib:", installed_packages.get("matplotlib"))
print("Scikit-learn:", installed_packages.get("scikit-learn"))

TensorFlow: 2.15.0
OpenCV: 4.11.0.86
NumPy: 1.24.3
Matplotlib: 3.10.5
Scikit-learn: 1.7.1


In [3]:
import warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt 
from keras import Input

# Dataset Preparation

In [4]:
idg=ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [5]:
# Train Dataset 
train_data=idg.flow_from_directory(
    r"C:\Users\SAMA\Downloads\Sign-Language-Detection\Dataset\images for phrases",
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical',
    subset='training',
)

Found 1408 images belonging to 44 classes.


In [6]:
# Test dataset 
test_data=idg.flow_from_directory(
    r"C:\Users\SAMA\Downloads\Sign-Language-Detection\Dataset\images for phrases",
    batch_size=32,
    target_size=(64,64),
    class_mode='categorical',
    subset='validation',
    shuffle=False , # <-- IMPORTANT
)

Found 352 images belonging to 44 classes.


# Build the CNN Model

In [7]:
model=Sequential([
    Input(shape=(64, 64, 3)),
    Conv2D(32,(3,3),activation='relu'), #Input layer
    MaxPooling2D(2,2), # layer 2 

    Conv2D(64,(3,3),activation='relu'), #Layer 3
    MaxPooling2D(2,2), # layer 4
    
    Flatten(),#layer5
    Dense(128,activation='relu'), #layer 6
    Dropout(0.3), #layer 7
    Dense(train_data.num_classes,activation='softmax'), # output layer
    
])
#all layers bw input and output are hidden layers

# Compile the Model

In [8]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model

In [17]:
model.fit(train_data,validation_data=test_data,epochs=20)

C:\Users\SAMA\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.1936 - loss: 3.2640 - val_accuracy: 0.6193 - val_loss: 1.6876
Epoch 2/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 31s 682ms/step - accuracy: 0.7340 - loss: 0.9472 - val_accuracy: 0.9403 - val_loss: 0.3255
Epoch 3/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 34s 765ms/step - accuracy: 0.9454 - loss: 0.2382 - val_accuracy: 0.9659 - val_loss: 0.1735
Epoch 4/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 32s 704ms/step - accuracy: 0.9773 - loss: 0.1096 - val_accuracy: 0.9744 - val_loss: 0.1493
Epoch 5/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 35s 777ms/step - accuracy: 0.9773 - loss: 0.0859 - val_accuracy: 0.9688 - val_loss: 0.1497
Epoch 6/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 31s 694ms/step - accuracy: 0.9908 - loss: 0.0406 - val_accuracy: 0.9716 - val_loss: 0.1397
Epoch 7/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 28s 626ms/step - accuracy: 0.9950 - loss: 0.0204 - val_accuracy: 0.9716 - val_loss: 0.1735
Epoch 8/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 30s 658ms/step - accuracy: 0.9915 - loss: 0.0414 - val_accurac

# Accuracy score and Confusion Matrix

In [20]:
y_pred_probs = model.predict(test_data)
import numpy as np

y_pred = np.argmax(y_pred_probs, axis=1)  # predicted labels
y_true = test_data.classes  # actual labels
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_true, y_pred)
print(f"Accuracy: {acc*100:.2f}%")


11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 529ms/step
Accuracy: 98.30%


# Save the model 

In [ ]:
model.save('../models/sign_language_model.h5')